In [29]:
from functools import lru_cache
from openai import OpenAI
client = OpenAI(api_key=API_KEY)

In [30]:
# Function to call ChatGPT.

@lru_cache(maxsize=None)
def call_gpt(prompt):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
      "role": "user",
      "content": prompt
    }
    ],
    temperature=0.7,
    max_tokens=64,
    top_p=1
    )
    return response.choices[0].message.content

In [53]:
import types
import json
from jinja2 import Template

# The implementation of the stack-based language.

class StackLang:
    def __init__(self):
        self.stack = []

    def push(self, value):
        self.stack.append(value)

    def pop(self):
        if self.stack:
            return self.stack.pop()
        raise IndexError("Pop from an empty stack")

    def MAKE_PROMPT(self):
        if len(self.stack) >= 2:
            prompt_template = self.pop()
            params = self.pop()
            template = Template(prompt_template)
            self.push(template.render(params))

        else:
            raise IndexError("CALL-ing a model requires 2 params: prompt and model params.")

    def CALL(self):
        if len(self.stack) >= 2:
            model = self.pop()
            prompt = self.pop()
            print('%% Calling model %s with prompt: %s' %(model, prompt))
            r = call_gpt(prompt)
            self.push(r)
        else:
            raise IndexError("CALL-ing a model requires 2 params: prompt and model params.")

    def ADD2 (self):
        if len(self.stack) >= 2:
            dict1 = self.pop()
            dict2 = self.pop()
            self.push({**dict1, **dict2})
        else:
            raise IndexError("ADD2 requires 2 params on the stack")


    def JSON(self):
        if len(self.stack) >= 1:
            self.push(json.loads(self.pop()))
        else:
            raise IndexError("JSON requires one param on the stack")
                      

    def execute(self, commands):
        #print(self)

        for cmd in commands:
            print(self)
            if cmd.startswith('"""'):
                self.push(cmd)
            elif cmd.startswith('{'):
                self.push(json.loads(cmd))
            else:
                if hasattr(self, cmd):
                    getattr(self, cmd)()
                else:
                    raise ValueError(f"Unknown command: {cmd}")

    def __str__(self):
            return str(self.stack)

In [12]:
# Function to read the file describing the LLM "program". 

def process_file(filename):
    lines = open(filename).readlines()
    clean_lines = []
    i = 0
    while i < len(lines):
        if lines[i].startswith('--'):
            i = i + 1
            continue
        if lines[i].startswith('"""') is False:
            clean_lines.append(lines[i].strip())
            i = i + 1
        else:
            j = i
            while (j < len(lines) and lines[j].strip().endswith('"""') is False):
                j = j + 1
            # clean_lines.append("\n".join(  [k[:-1] for k in lines[i:j+1] ] ) )
            clean_lines.append("\n".join(lines[i:j+1]).strip())
            i = j + 1
    return [k for k in clean_lines if k != '']

In [54]:
# We run the program against one of the files.

lang = StackLang()
commands = process_file('llm2.txt')
lang.execute(commands)

print()

print(lang)


[]
[{'habitat': 'ocean'}]
[{'habitat': 'ocean'}, '"""pick a random animal from the {{habitat}}. Return just the name, as json, using `animal`."""']
['"""pick a random animal from the ocean. Return just the name, as json, using `animal`."""']
['"""pick a random animal from the ocean. Return just the name, as json, using `animal`."""', {'model': 'gpt-3.5', 'temperature': 0.8}]
% Calling model {'model': 'gpt-3.5', 'temperature': 0.8} with prompt: """pick a random animal from the ocean. Return just the name, as json, using `animal`."""
['{\n    "animal": "jellyfish"\n}']
[{'animal': 'jellyfish'}]
[{'animal': 'jellyfish'}, {'genre': 'princess'}]
[{'animal': 'jellyfish'}, {'genre': 'princess'}, '"""Pick a random fictional character from children\'s book talkinbg about {{genre}}. Return just the name, as json, using `character`."""']
[{'animal': 'jellyfish'}, '"""Pick a random fictional character from children\'s book talkinbg about princess. Return just the name, as json, using `character`."